In [15]:
import pandas as pd
import os

from maca import clean_annotation, clean_labels

metadata_folder = os.path.join('..', 'metadata' )

annotation_folder = os.path.join(metadata_folder, 'manual_annotations')
# annotation_folder

summary_folder = os.path.join(metadata_folder, 'summary')
! mkdir $summary_folder

%load_ext autoreload
%autoreload 2


mkdir: ../metadata/summary: File exists
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import glob
import os

dfs = []

original_dfs = []

skip_tissues = [] #['Trachea',]

globber = os.path.join(annotation_folder, '*', '*.csv')

for csv in glob.glob(globber):
    if '10x' in csv.lower():
        continue
    basename = os.path.basename(csv)
    tissue = os.path.dirname(csv).split('/')[-1]
        
    print('\n-', tissue,  '-', basename)
    if tissue == "Lung":
        df = pd.read_csv(csv, usecols=["V1", "V2"], index_col=0)
        df.columns = ['annotation']
#         brea
    else:
        df = pd.read_csv(csv, index_col=0)
#     print('\t', '\n\t'.join(df['annotation'].astype(str).unique()))
    try:
        sizes = df.fillna('.').groupby(['annotation', 'subannotation']).size()
    except KeyError:
        try:
            sizes = df.fillna('.').groupby(['annotation']).size()
            print('-- before cleaning --\n', )
        except KeyError:
            sizes = df.head()
            print('-- no "annotation" or "subannotation" columns --', )
            
    print('-- before cleaning --\n', sizes)
    original_dfs.append(df)
    
    df = clean_annotation(df, tissue, debug=True)
    sizes = df.fillna('..').groupby(['annotation', 'subannotation']).size()
    print('--- after cleaning ---\n', sizes)
    df['tissue'] = tissue
    
    if tissue not in skip_tissues:
        dfs.append(df)
    
    
combined = pd.concat(dfs)

# Remove Heart annotation of Aorta cells. 
# "Heart" comes after "Aorta" alphabetically so we're using 
# "drop_duplicates" which keeps only the first occurence
combined = combined.loc[~combined.index.duplicated()]

combined['annotation_subannotation'] = combined.apply(
    lambda x: '{annotation}: {subannotation}'.format(**x) 
    if pd.notnull(x['subannotation']) else x['annotation'], axis=1)

combined.head()


- Aorta - Heart_annotation.csv
-- before cleaning --
 annotation                   subannotation               
adipocytes                   .                                54
endothelial cells            .                               131
fibroblasts                  .                                35
hematopoetic cells           .                                17
heterogenous group of cells  adipocytes                       39
                             erythroblasts and adipocytes     51
                             smooth muscle cells              37
dtype: int64
		--- after cleaning labels ---
annotation                   subannotation               
adipocytes                   -                                54
endothelial_cells            -                               131
fibroblasts                  -                                35
hematopoetic_cells           -                                17
heterogenous_group_of_cells  adipocytes                       39
    

--- after cleaning ---
 annotation               subannotation
b_cells                  ..                513
endothelial              ..                590
epithelial               ..                 51
granulocytes             ..                409
multipotent_progenitors  ..               1924
muscle                   ..                 27
myeloid                  ..                479
natural_killer           ..                129
neutrophils              ..                221
t_cells                  ..                366
dtype: int64

- Heart - Heart_metadata.csv
-- before cleaning --
 annotation               subannotation      
CMs                      .                      143
Edc_1                    .                      894
Edc_2_coronary_vascular  .                      326
Edc_3_endocardial        .                      182
Edc_4_aorta              .                       80
Fb_1                     .                      993
Fb_2                     .                  

-- before cleaning --
 annotation                        subannotation        
basal cells                       .                         37
                                  1                        318
                                  2                        233
                                  Chac1+Mthfd2+            390
                                  Cxcl2+Vcam+Procr+         57
                                  Fos+                     111
                                  Id1+Id2+Pdfgrl+Wnt10a     52
                                  Ifgbp2+Igfbp4             77
endothelial cells                 .                         49
hormone responsive luminal cells  .                        156
luminal progenitors               .                        396
stromal cells                     .                        428
dtype: int64
		--- after cleaning labels ---
annotation                        subannotation        
basal_cells                                                551
 

dtype: int64
--- after cleaning ---
 annotation                 subannotation         
hair_follicle              inner_bulge               824
                           outer_bulge               547
interfollicular_epidermis  basal_ife_cells           402
                           intermediate_ife_cells    259
                           outer_ife_cells           186
                           proliferating              45
dtype: int64

- Spleen - Spleen_annotation.csv
-- before cleaning --
 annotation                                        subannotation
9CD8+ T cells                                     .                  81
CD4+ T cells                                      .                 273
Macrophages/Dendritic cells/Monocytes and others  .                  48
Marginal Zone B cells                             .                 153
Natural Killer cells                              .                  49
T1/T2/Follicular B cells                          .                1085
dtype

,annotation,plate.barcode,subannotation,tissue,annotation_subannotation
A21.MAA000594.3_8_M.1.1,fibroblasts,maa,NaN,Aorta,fibroblasts
F8.MAA000594.3_8_M.1.1,unknown,maa,NaN,Aorta,unknown
H11.MAA000594.3_8_M.1.1,unknown,maa,NaN,Aorta,unknown
A22.MAA000594.3_8_M.1.1,unknown,maa,NaN,Aorta,unknown
H12.MAA000594.3_8_M.1.1,adipocytes,maa,NaN,Aorta,adipocytes


In [17]:
combined['plate.barcode'] = combined.index.map(lambda x: x.split('.')[1])
combined.head()

,annotation,plate.barcode,subannotation,tissue,annotation_subannotation
A21.MAA000594.3_8_M.1.1,fibroblasts,MAA000594,NaN,Aorta,fibroblasts
F8.MAA000594.3_8_M.1.1,unknown,MAA000594,NaN,Aorta,unknown
H11.MAA000594.3_8_M.1.1,unknown,MAA000594,NaN,Aorta,unknown
A22.MAA000594.3_8_M.1.1,unknown,MAA000594,NaN,Aorta,unknown
H12.MAA000594.3_8_M.1.1,adipocytes,MAA000594,NaN,Aorta,adipocytes


In [18]:
original_annotations = pd.concat(original_dfs)
print(original_annotations.shape)

# Drop the double-annotated heart and aorta annotations
original_annotations = original_annotations.loc[~original_annotations.index.duplicated()]
print(original_annotations.shape)
original_annotations = original_annotations.drop('plate.barcode', axis=1)
original_annotations.head()

combined_with_original = combined.join(original_annotations, rsuffix='_original', lsuffix='_cleaned')
print(combined_with_original.shape)
# combined_with_original.head()

cols = ['tissue', 'annotation_original', 'subannotation_original', 'annotation_cleaned', 'subannotation_cleaned',]

# combined_with_original.fillna('.').groupby(['tissue', 'annotation', 'subannotation', 'annotation_original', 'subannotation_original']).size()

combined_sizes = combined_with_original.fillna('.').groupby(cols).size().reset_index()
combined_sizes = combined_sizes.rename(columns={0: 'n_cells'})
print(combined_sizes.shape)
combined_sizes.head()

from IPython.display import display, HTML

for tissue, df in combined_sizes.groupby('tissue'):
    display(df)

(42556, 3)
(42192, 3)
(42192, 7)
(211, 6)


,tissue,annotation_original,subannotation_original,annotation_cleaned,subannotation_cleaned,n_cells
0,Aorta,adipocytes,.,adipocytes,.,54
1,Aorta,endothelial cells,.,endothelial,.,131
2,Aorta,fibroblasts,.,fibroblasts,.,35
3,Aorta,hematopoetic cells,.,hematopoietic,.,17
4,Aorta,heterogenous group of cells,adipocytes,adipocytes,.,39
5,Aorta,heterogenous group of cells,erythroblasts and adipocytes,unknown,.,51
6,Aorta,heterogenous group of cells,smooth muscle cells,smooth_muscle,.,37


,tissue,annotation_original,subannotation_original,annotation_cleaned,subannotation_cleaned,n_cells
7,Bladder,Basal,.,basal,.,99
8,Bladder,LuminalA1,.,luminal,a,191
9,Bladder,LuminalA2,.,luminal,a,245
10,Bladder,LuminalB,.,luminal,b,96
11,Bladder,MesenchymalA1,.,mesenchymal,a,170
12,Bladder,MesenchymalA2,.,mesenchymal,a,165
13,Bladder,MesenchymalB1,.,mesenchymal,b,156
14,Bladder,MesenchymalB2,.,mesenchymal,b,165


,tissue,annotation_original,subannotation_original,annotation_cleaned,subannotation_cleaned,n_cells
15,Brain_Microglia,Macrophages,.,macrophages,.,36
16,Brain_Microglia,Microglia,.,microglia,.,4329


,tissue,annotation_original,subannotation_original,annotation_cleaned,subannotation_cleaned,n_cells
17,Brain_Neurons,NPCS,.,npcs,.,36
18,Brain_Neurons,OPCs,.,opcs,.,202
19,Brain_Neurons,astrocytes,Berg.Glia,astrocytes,berg_glia,30
20,Brain_Neurons,astrocytes,cerebral_astrocytes,astrocytes,cerebral_astrocytes,403
21,Brain_Neurons,astrocytes,myelinating_oligodendrocytes,astrocytes,myelinating_oligodendrocytes,11
22,Brain_Neurons,endothelial,vSMCs,endothelial,vsmcs,14
23,Brain_Neurons,endothelial,vascular,endothelial,vascular,625
24,Brain_Neurons,neurons,doublet,neurons,doublet,2
25,Brain_Neurons,neurons,excitatory,neurons,excitatory,73
26,Brain_Neurons,neurons,inhibitory,neurons,inhibitory,98


...

,tissue,annotation_original,subannotation_original,annotation_cleaned,subannotation_cleaned,n_cells
179,Skin,Basal IFE,.,interfollicular_epidermis,basal_ife_cells,402
180,Skin,Cell Cycle,.,interfollicular_epidermis,proliferating,45
181,Skin,Inner Bulge,.,hair_follicle,inner_bulge,824
182,Skin,Intermediate IFE,.,interfollicular_epidermis,intermediate_ife_cells,259
183,Skin,Outer Bulge,.,hair_follicle,outer_bulge,547
184,Skin,Outer IFE,.,interfollicular_epidermis,outer_ife_cells,186


,tissue,annotation_original,subannotation_original,annotation_cleaned,subannotation_cleaned,n_cells
185,Spleen,9CD8+ T cells,.,t_cells,cd8+,81
186,Spleen,CD4+ T cells,.,t_cells,cd4+,273
187,Spleen,Macrophages/Dendritic cells/Monocytes and others,.,myeloid,.,48
188,Spleen,Marginal Zone B cells,.,b_cells,marginal_zone,153
189,Spleen,Natural Killer cells,.,t_cells,natural_killer_cells,49
190,Spleen,T1/T2/Follicular B cells,.,b_cells,follicular,1085


,tissue,annotation_original,subannotation_original,annotation_cleaned,subannotation_cleaned,n_cells
191,Thymus,stromal_mesenchymal stem cell,.,stromal_stem,.,33
192,Thymus,thymocyte_1_mix of DN4_DP_immatureSPs,DN4,t_cells,double_negative4,92
193,Thymus,thymocyte_1_mix of DN4_DP_immatureSPs,DP,t_cells,double_positive,44
194,Thymus,thymocyte_1_mix of DN4_DP_immatureSPs,Immature_SP_CD8,t_cells,immature_single_positive_cd8+,62
195,Thymus,thymocyte_1_mix of DN4_DP_immatureSPs,immature_SP_CD4,t_cells,immature_single_positive_cd4+,280
196,Thymus,thymocyte_2_DP_favoring CD8 differentiation,.,t_cells,double_positive_favoring_cd8+,323
197,Thymus,thymocyte_3_DP_favoring CD8 differentation,.,t_cells,double_positive_favoring_cd8+,314
198,Thymus,thymocyte_4_DP_rapidly dividing_favoring CD8 d...,.,t_cells,double_positive_rapidly_dividing_favoring_cd8+,103
199,Thymus,thymocyte_5_DN1,.,t_cells,double_negative1,32


,tissue,annotation_original,subannotation_original,annotation_cleaned,subannotation_cleaned,n_cells
200,Tongue,Basal/differentiating,.,basal,basal,831
201,Tongue,Filiform,.,keratinocytes,filiform_cells,35
202,Tongue,Maturing/nonkeratinized,.,keratinocytes,suprabasal,178
203,Tongue,Proliferating,.,basal,proliferating,202
204,Tongue,Stratified/differentiated keratinocytes,.,keratinocytes,suprabasal_cells,148


,tissue,annotation_original,subannotation_original,annotation_cleaned,subannotation_cleaned,n_cells
205,Trachea,Epcam,Krt5,epithelial,basal_cells,29
206,Trachea,Epcam,No Krt5,epithelial,no_krt5,44
207,Trachea,Pdgfrb,.,stromal,.,569
208,Trachea,Pecam1,.,endothelial,.,33
209,Trachea,Ptprc,.,immune,.,116
210,Trachea,Scgb1a1,.,epithelial,secretory_cells,55


Write combined original and cleaned

In [19]:
csv = os.path.join(summary_folder, 'maca_3month_original_vs_cleaned_plates.csv')
combined_sizes.to_csv(csv, index=False)

## make sure there are no duplicated cell ids

In [20]:
combined.index.duplicated().sum()

0

## Make sure there are 20 tissues

In [21]:
tissues = sorted(combined['tissue'].unique())
tissues

['Aorta',
 'Bladder',
 'Brain_Microglia',
 'Brain_Neurons',
 'Colon',
 'Diaphragm',
 'Fat',
 'Heart',
 'Kidney',
 'Liver',
 'Lung',
 'Mammary',
 'Marrow',
 'Muscle',
 'Pancreas',
 'Skin',
 'Spleen',
 'Thymus',
 'Tongue',
 'Trachea']

In [22]:
pd.Series(tissues)

0               Aorta
1             Bladder
2     Brain_Microglia
3       Brain_Neurons
4               Colon
5           Diaphragm
6                 Fat
7               Heart
8              Kidney
9               Liver
10               Lung
11            Mammary
12             Marrow
13             Muscle
14           Pancreas
15               Skin
16             Spleen
17             Thymus
18             Tongue
19            Trachea
dtype: object

    0                    Aorta
    1                  Bladder
    2     Brain_FACS_microglia
    3       Brain_FACS_neurons
    4                    Colon
    5                Diaphragm
    6                      Fat
    7                    Heart
    8                   Kidney
    9                    Liver
    10                    Lung
    11           Mammary_Gland
    12                  Marrow
    13                  Muscle
    14                Pancreas
    15                    Skin
    16                  Spleen
    17                  Thymus
    18                  Tongue
    19                 Trachea
    dtype: object

In [23]:
combined.groupby('tissue').size()


tissue
Aorta               364
Bladder            1287
Brain_Microglia    4365
Brain_Neurons      3000
Colon              3459
Diaphragm           870
Fat                4709
Heart              4221
Kidney              517
Liver               710
Lung               1620
Mammary            2304
Marrow             4897
Muscle             1067
Pancreas           1327
Skin               2263
Spleen             1689
Thymus             1283
Tongue             1394
Trachea             846
dtype: int64

In [24]:
pd.options.display.max_rows = 100

In [25]:
annotation_sizes = combined.groupby('annotation').size().sort_values(ascending=False).to_frame()
annotation_sizes = annotation_sizes.rename(columns={0: 'n_cells'})
print(annotation_sizes.shape)
annotation_sizes

(65, 1)


,n_cells
annotation,
microglia,4329
b_cells,4011
endothelial,3983
basal,2407
t_cells,2251
fibroblasts,2152
multipotent_progenitors,1924
hair_follicle,1371
stromal,1363


In [26]:
csv = os.path.join(summary_folder, 'maca_3month_annotation_counts_plates.csv')
annotation_sizes.to_csv(csv)

In [27]:
len(combined.groupby('tissue').size())

20

### Write plate annotation

In [28]:
csv = os.path.join(metadata_folder, 'maca_3month_annotations_plates.csv')

combined.to_csv(csv)